<a href="https://colab.research.google.com/github/deniskapel/topic_rec/blob/master/topic_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Next entity recommendation using Spacy noun chunks and LSTM

Steps:
1. Extract noun chunks using Spacy
2. Filter out most frequent
3. Cluster dialogues using chunks vector and Kmeans
4. Take the largest cluster and split it to train and test
5. Train LSTM model to generate next entity (noun chunk)

In [ ]:
# skip if you are using jupyter instead of colab. GPU is required.
%%bash
wget https://raw.githubusercontent.com/deniskapel/topic_rec/master/data_tools.py
wget https://raw.githubusercontent.com/deniskapel/topic_rec/master/torch_tools.py

In [2]:
import numpy as np
import spacy
import torch

from torch.utils.data import DataLoader
from sklearn.cluster import KMeans

from data_tools import Chunker, Vectorizer, SequenceGenerator
from torch_tools import TopicDataset, GModel, train, predict

In [3]:
spacy.prefer_gpu()
# load larger model to use spacy word vectors
spacy.cli.download("en_core_web_md")
nlp = spacy.load("en_core_web_md")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [4]:
device = torch.device('cuda')

## Data

In [ ]:
# skip if data is loaded before
# %%bash
# wget http://yanran.li/files/ijcnlp_dailydialog.zip
# unzip ijcnlp_dailydialog.zip
# mv ijcnlp_dailydialog data

In [6]:
"""
work only with a train set only as we were supposed to filter out topics 
using key words not topic labels
"""
with open('data/dialogues_text.txt') as file:
    dialogues = file.readlines()

In [7]:
dialogues[0]

"The kitchen stinks . __eou__ I'll throw out the garbage . __eou__\n"

### Normalizing data

In [8]:
chunker = Chunker(spacy_model=nlp, stop_words=nlp.Defaults.stop_words)
chunked_dials = chunker.normalize(dialogues[0:1000])

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
chunked_dials[90]

[Ve,
 your computer,
 Ve,
 your personal files,
 ’ m,
 all my important personal documents,
 that computer,
 no laughing matter,
 I ’ m,
 Don ’ t,
 my computer,
 t]

remove 100 most common noun chunks as they do not characterize dialogues

**TODO**: experiment with larger/smaller number and a stop list

In [9]:
most_common = [topic[0] for topic in chunker.counter.most_common(100)]
filtered = [chunker.filter_chunks(dial, most_common) for dial in chunked_dials]
set(filtered[90]) ^ set(chunked_dials[90])

{t}

In [10]:
# filter out dialogues which have less than three noun chunks
# for prediction, I will use two previous noun chunks
filtered = [doc for doc in filtered if len(doc) > 1]

### Vectorizing data to cluster

In [11]:
vectorizer = Vectorizer(len(filtered))
vecs, doc2id = vectorizer.vectorize(filtered)

In [12]:
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(vecs)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [13]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
topics = dict(zip(unique, counts))
topics

{0: 62, 1: 236, 2: 168, 3: 261, 4: 137}

In [14]:
id_max = max(topics, key=topics.get)
id_max

3

Take the largest cluster to have data for training

In [15]:
largest_theme = doc2id.loc[(kmeans.labels_== id_max).nonzero()]

In [17]:
largest_theme.sample(5)

,doc
718,"[(a, pet), (Dogs), (enough, exercise), (a, dog..."
729,"[(the, well, -, done, steak), (a, mistake)]"
66,"[(this, movie), (Just, 10, more, minutes)]"
110,"[(a, room), (enough, space), (your, own, room)..."
72,"[(the, bedroom), (This, afternoon)]"


Build X and y for training

In [18]:
sgen = SequenceGenerator(1)
seqs = sgen.get_sequences(largest_theme)
seqs.head()

,seq,target
0,1957,577
1,577,1514
2,1514,1764
3,1764,898
4,898,290


train test split

In [19]:
train_df=seqs.sample(frac=0.8,random_state=42)
test_df=seqs.drop(train_df.index)

In [20]:
train_df.shape, test_df.shape

((1489, 2), (372, 2))

## Data to torch DataLoader

In [21]:
# lstm accepts hidden_state and cell_state of the same size
# to avoid initiating it at each batch, make all batches of equal size
# alternative solution is to add up a few examples from validation
BSize = 4
trim_train = train_df.shape[0] % BSize
trim_test = test_df.shape[0] % BSize

train_x = train_df['seq'].values
train_y = train_df['target'].values
test_x = test_df['seq'].values
test_y = test_df['target'].values
if trim_train>0:
    train_x = train_x[:-trim_train]
    train_y = train_y[:-trim_train]
if trim_test>0:
    test_x = test_x[:-trim_test]
    test_y = test_y[:-trim_test]

train_dataset = TopicDataset(
    x=train_x,
    y=train_y,
    n_features=300,
    id2chunk=sgen.id2chunk,
    chunk2id=sgen.chunk2id,
    seq_len=sgen.seq_len)

test_dataset = TopicDataset(
    x=test_x,
    y=test_y,
    n_features=300,
    id2chunk=sgen.id2chunk,
    chunk2id=sgen.chunk2id,
    seq_len=sgen.seq_len)

In [22]:
# vector size               target                  prev_seq
train_dataset[0][0].shape, train_dataset[0][1],train_dataset[0][2]

((300,), 276, 438)

In [23]:
# vector size               target                  prev_seq
test_dataset[0][0].shape, test_dataset[0][1], test_dataset[0][2]

((300,), 1514, 577)

Split the dataset to train, val and test

In [24]:
train_loader = DataLoader(
    train_dataset, batch_size=BSize, 
    shuffle=True, collate_fn=train_dataset.collate,
    # TODO find out why shuffling requires a generator on gpu 
    # probably due to spacy vectors on gpu,
    # but shuffle=False works fine without it for some reason
    generator=torch.Generator(device='cuda'))

for batch in train_loader:
    break

batch[0].shape

torch.Size([4, 1, 300])

In [25]:
test_loader = DataLoader(
    test_dataset, batch_size=BSize, 
    shuffle=True, collate_fn=test_dataset.collate,
    generator=torch.Generator(device='cuda'))

In [26]:
for batch in test_loader:
    break

batch[1]

tensor([ 224, 1308,  253,  466])

## Train

use lstm to generate next topic

**TODO:** experiment with number of epochs, various combinations of layers, larger/smaller sequence

**TODO2:** refactor to use any sequence length to predict the next entity

In [27]:
model = GModel(vocab_size=len(sgen.id2chunk), seq_len=sgen.seq_len)

In [28]:
model.to(device)

GModel(
  (lstm): LSTM(300, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=2122, bias=True)
)

In [29]:
train(model, train_loader, epochs=10, lr=.001, clip_value=1.0)

Epoch 1:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 3:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 4:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 5:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 6:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 7:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 8:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 9:   0%|          | 0/372 [00:00<?, ?it/s]

Epoch 10:   0%|          | 0/372 [00:00<?, ?it/s]

Prediction is using the first batch only for demosntration purposes. Data is shuffled, so output is always different

In [31]:
predict(model, test_loader, sgen.id2chunk)

After --a happy woman--, I suggest discussing --a leave--
After --cinemas--, I suggest discussing --a system--
After --Good idea--, I suggest discussing --any change--
After --Uh ! Dad--, I suggest discussing --computer games--


In [32]:
predict(model, test_loader, sgen.id2chunk)

After --a walk--, I suggest discussing --a walk--
After --darling--, I suggest discussing --our clarifying cream--
After --the usual sale price--, I suggest discussing --a good system--
After --food--, I suggest discussing --your brother--
